In [6]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

# Load dataset
digits = load_digits()

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.3, random_state=42)


In [7]:
from skimage.feature import hog
from skimage.transform import resize

# Resize input images to (64, 64)
X_train_resized = [resize(image, (64, 64)) for image in X_train]
X_test_resized = [resize(image, (64, 64)) for image in X_test]

# Extract HOG features from images
X_train_hog = []
for image in X_train_resized:
    hog_features = hog(image, block_norm='L2-Hys')
    X_train_hog.append(hog_features)

X_test_hog = []
for image in X_test_resized:
    hog_features = hog(image, block_norm='L2-Hys')
    X_test_hog.append(hog_features)

In [8]:
X_train_hog==X_test_hog

False

In [9]:
from sklearn.tree import DecisionTreeClassifier

# Train multiple decision trees as weak classifiers
weak_classifiers = []
for i in range(5):
    dtc = DecisionTreeClassifier(max_depth=3, random_state=42)
    dtc.fit(X_train_hog, y_train)
    weak_classifiers.append(dtc)

In [10]:
from sklearn.ensemble import AdaBoostClassifier

# Train AdaBoostClassifier using weak classifiers
ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3, random_state=42),
                         n_estimators=5,
                         algorithm='SAMME.R',
                         learning_rate=0.5,
                         random_state=42)
ada.fit(X_train_hog, y_train)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3,
                                                         random_state=42),
                   learning_rate=0.5, n_estimators=5, random_state=42)

In [11]:
# Combine predictions of weak classifiers to make final prediction
y_pred = ada.predict(X_test_hog)

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Evaluate performance of AdaBoostClassifier
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred, average='weighted'))
print('Recall:', recall_score(y_test, y_pred, average='weighted'))
print('F1-score:', f1_score(y_test, y_pred, average='weighted'))

Accuracy: 0.12222222222222222
Precision: 0.39988880621693124
Recall: 0.12222222222222222
F1-score: 0.07039398591477806


/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
